# Influência do tamanho do batch

In [1]:
import seaborn as sns

In [2]:
titanic_df = sns.load_dataset('titanic')

In [3]:
feature_names = ['pclass', 'female', 'age', 'fare'] # classe, feminino, idade, preço da passagem
titanic_df['female'] = titanic_df['sex'].map({'male': 0, 'female':1})
titanic_df['alive'] = titanic_df['alive'].map({'no': 0, 'yes':1})
titanic_df.dropna(subset=feature_names, inplace=True)

In [4]:
X = titanic_df[feature_names].to_numpy()
Y = titanic_df['alive'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [7]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.linear1 = nn.Linear(4, 4)    # primeira hidden layer
        self.dropout1 = nn.Dropout(0.2)   # dropout layer
        self.linear2 = nn.Linear(4, 4)    # segunda hidden layer
        self.dropout2 = nn.Dropout(0.2)   # dropout layer
        self.linear3 = nn.Linear(4, 1)    # terceira hidden layer
        self.dropout3 = nn.Dropout(0.2)   # dropout layer
        self.sigmoid = nn.Sigmoid()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        x = F.relu(self.linear3(x))
        x = self.dropout3(x)
        x = self.sigmoid(x)
        return x

model = ClassBin()

In [ ]:
# class ClassBin(nn.Module):
#     # Construtor
#     def __init__(self):
#         super(ClassBin, self).__init__()
#         self.fc1 = nn.Linear(4, 20) # primeira hidden layer
#         self.fc2 = nn.Linear(20, 1) # segunda hidden layer
#         self.sigmoid = nn.Sigmoid() # output layer com ativação Sigmoid

#     # Propagação (Feed Forward)
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.sigmoid(x)
#         return x

# model = ClassBin()

In [8]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 4  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.1

# Instânciar o Otimizador Adam
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
from torch.utils.data import TensorDataset, DataLoader

# Converter X e y para torch.Tensor
X_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test)

# Um Dataset de Tensores - Array [X, y]
train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [10]:
for t in range(epochs):
    model.train() # Colocar o modelo em modo de treinamento (calcula os gradientes)
    
    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]
        
        # Propagação (Feed Forward)
        y_pred = model(X)
    
        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))
        
        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.693
Época 2,
          Custo Treino: 0.693
Época 3,
          Custo Treino: 0.693
Época 4,
          Custo Treino: 0.693
Época 5,
          Custo Treino: 0.693
Época 6,
          Custo Treino: 0.693
Época 7,
          Custo Treino: 0.693
Época 8,
          Custo Treino: 0.693
Época 9,
          Custo Treino: 0.693
Época 10,
          Custo Treino: 0.693
Época 11,
          Custo Treino: 0.693
Época 12,
          Custo Treino: 0.693
Época 13,
          Custo Treino: 0.693
Época 14,
          Custo Treino: 0.693
Época 15,
          Custo Treino: 0.693
Época 16,
          Custo Treino: 0.693
Época 17,
          Custo Treino: 0.693
Época 18,
          Custo Treino: 0.693
Época 19,
          Custo Treino: 0.693
Época 20,
          Custo Treino: 0.693
Época 21,
          Custo Treino: 0.693
Época 22,
          Custo Treino: 0.693
Época 23,
          Custo Treino: 0.693
Época 24,
          Custo Treino: 0.693
Época 25,
          Custo Treino: 0.693
Época 26,

In [11]:
model.eval()
    
with torch.no_grad():  # Desabilitar gradientes durante avaliação
    train_pred = model(X_train)
    train_pred_binary = (train_pred > 0.5).float().flatten()
    train_acc = torch.sum(train_pred_binary == y_train) / len(y_train)

    test_pred = model(X_test)
    test_pred_binary = (test_pred > 0.5).float().flatten()
    test_acc = torch.sum(test_pred_binary == y_test) / len(y_test)

print(f"Acurácia de Treino: {train_acc:.6f}")
print(f"Acurácia de Teste: {test_acc:.6f}")

Acurácia de Treino: 0.593695
Acurácia de Teste: 0.594406
